## Стихи Пушкина

In [8]:
!pip install datasets


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch

from torch.nn import functional as F

In [12]:
from datasets import load_dataset

raw_datasets = load_dataset("abobster/pushkin_new")

with open('input.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_datasets['train']['text']))

In [39]:
!cat input.txt



Но грустно думать, что напрасно
Была нам молодость дана,
Что изменяли ей всечасно,
Что обманула нас она;
Что наши лучшие желанья,
Что наши свежие мечтанья
Истлели быстрой чередой,
Как листья осенью гнилой.
Несносно видеть пред собою
Одних обедов длинный ряд,
Глядеть на жизнь, как на обряд,
И вслед за чинною толпою
Идти, не разделяя с ней
Ни общих мнений, ни страстей
Предметом став суждений шумных,
Несносно согласитесь в том
Между людей благоразумных
Прослыть притворным чудаком,
Или печальным сумасбродом,
Иль сатаническим уродом,
Иль даже демоном моим.
Онегин вновь займуся им,
Убив на поединке друга,
Дожив без цели, без трудов
До двадцати шести годов,
Томясь в бездействии досуга
Без службы, без жены, без дел,
Ничем заняться не умел.

</s>

Менко Вуич грамоту пишет
Своему побратиму:
«Берегися, Черный Георгий,
Над тобой подымается туча,
Ярый враг извести тебя хочет,
Недруг хитрый, Милош Обренович.
Он в Хотин подослал потаенно
Янка младшего с Павлом
Осердился Георгий Петрович,
Засверкали

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print('device', device)

device mps


In [4]:
batch_size = 64  # independent sequences processed in parallel
block_size = 256  # maximum context length for predictions
max_iters = 5000
eval_interval = 50
learning_rate = 3e-4

eval_iters = 200
n_embd = 384
dropout = 0.2

In [69]:
torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# todo: remove rare symbols
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('vocab_size', vocab_size)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

vocab_size 131


## Модель

In [98]:
import torch.nn as nn


class ModelRNN(nn.Module):
    def __init__(
            self,
            vocab_size,
            hidden_size,
            dropout,
            num_layers=1,
    ):
        super(ModelRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        output, hidden = self.lstm(x, hidden)
        output = self.dropout(output)
        output = self.fc(output)

        return output, hidden


In [99]:
model = ModelRNN(
    vocab_size=vocab_size,
    hidden_size=n_embd,
    dropout=dropout,
)
m = model.to(device)

print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

1.283459 M parameters


In [100]:
from time import time


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)

    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, _ = model(X)
            loss = F.cross_entropy(logits.view(-1, vocab_size), Y.view(-1))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [101]:
from utils import cur_dir

save_path = cur_dir() + '/rnnmodel.pth'

In [102]:
from tqdm import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

t0 = time()
losses = []
for i in tqdm(range(max_iters)):
    # todo: сделать 2 графика с трейн/вал, посмотреть, когда будет переобучение
    # if i % eval_interval == 0 or i == max_iters - 1:
    #     cur_loss = estimate_loss()
    #     losses.append(cur_loss)
    #     print(f"step {iter}: train loss {cur_loss['train']:.4f}, val loss {cur_loss['val']:.4f}, elapsed: {time() - t0:.1f}s")

    xb, yb = get_batch('train')
    logits, _ = model(xb, hidden=None)
    loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1))

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

torch.save(model.state_dict(), save_path)

100%|██████████| 5000/5000 [03:43<00:00, 22.41it/s]


In [ ]:
# model.load_state_dict(torch.load(save_path))

In [104]:
def generate(model, seed_text, tokens_cnt):
    model.eval()

    generated = seed_text.clone()
    for _ in tqdm(range(tokens_cnt)):
        logits, _ = model(generated)
        last_logits = logits[:, -1, :]
        probabilities = F.softmax(last_logits, dim=-1)
        sampled_token = torch.multinomial(probabilities, 1)
        generated = torch.cat((generated, sampled_token), dim=1)

    return generated

In [113]:
seed_text = 'О вы, которые любовью не горели'
print(f'Seed text: {seed_text}')

context = torch.tensor(encode(seed_text), dtype=torch.long, device=device).unsqueeze(0)

tokens_to_generate = 200
generated = generate(model, context, tokens_to_generate)

generated_text = decode(generated[0].tolist())

print(generated_text)

Seed text: О вы, которые любовью не горели


100%|██████████| 200/200 [00:00<00:00, 305.52it/s]

О вы, которые любовью не горелиги протсь руменам Какикрм та,
Жей,
Пим 
</s>
Тый ми Несей чи кобъена мн,
Он — ий утст Праемный ун?
Ви,
Пе, по дваный
За — во, вдмноюбияледи м воку оча ететая витеж —
Мны.
И ртоеерафусера ущоит.

Неть 


In [31]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install lxml

!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.1
    Uninstalling setuptools-65.5.1:
      Successfully uninstalled setuptools-65.5.1
  Using cached lxml-5.0.0.zip (4.1 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for lxml: filename=lxml-5.0.0-cp39-cp39-macosx_10_9_universal2.whl size=3210317 sha256=9065d967a4a52801700ab9981e41fe0c731b4d89ee2614af093151253560344c
  Stored in directory: /Users/timoniche/Library/Caches/pip/wheels/b3/f3/c8/7d32062bb4d205cfc564ff338613b965f852cfb428224a1e96
Successfully built lxml


In [115]:
references = [
    '''
    Лагерь при Евфрате
    Не пленяйся бранной славой,
    О красавец молодой!
    Не бросайся в бой кровавый
    С карабахскою толпой!
    Знаю, смерть тебя не встретит:
    Азраил, среди мечей,
    Красоту твою заметит —
    И пощада будет ей!
    Но боюсь: среди сражений
    Ты утратишь навсегда
    Скромность робкую движений,
    Прелесть неги и стыда! 
    ''',
    '''
    О вы, которые любовью не горели,
    Взгляните на нее — узнаете любовь.
    О вы, которые уж сердцем охладели,
    Взгляните на нее: полюбите вы вновь. 
    ''',
    '''
    Все кончено: меж нами связи нет.
    В последний раз обняв твои колени,
    Произносил я горестные пени.
    Все кончено — я слышу твой ответ.
    Обманывать себя не стану вновь,
    Тебя тоской преследовать не буду,
    Прошедшее, быть может, позабуду —
    Не для меня сотворена любовь.
    Ты молода: душа твоя прекрасна,
    И многими любима будешь ты. 
    ''',
]

In [122]:
seeds = [ref.splitlines()[1] for ref in references]

ref_lens = [len(ref) for ref in references]
seed_lens = [len(seed) for seed in seeds]
cnt_to_generate = [r_len - s_len for r_len, s_len in zip(ref_lens, seed_lens)]

print('Seed texts: ')
print(seeds)

Seed texts: 
['    Лагерь при Евфрате', '    О вы, которые любовью не горели,', '    Все кончено: меж нами связи нет.']


In [129]:
generated_texts = []
for i in range(len(seeds)):
    context = torch.tensor(encode(seeds[i]), dtype=torch.long, device=device).unsqueeze(0)
    tokens_to_generate = cnt_to_generate[i]
    generated = generate(model, context, tokens_to_generate)
    generated_text = decode(generated[0].tolist())
    generated_texts.append(generated_text)

    print(generated_text)

100%|██████████| 343/343 [00:01<00:00, 218.48it/s]


    Лагерь при Евфрате бе осо бусобе е в пая.
Я мескиреежо вых бел иж ноблм, т раяде енот вслю повалнаситет, могло…
</sintonz>
— татых ли — зннитныелыси,
Ты ник счивой ниценасемо Слоюз мидра сть убумиго мчь
Игогдим?
Геч по, ракетитоный,
Кедлиное ге ст
Я Вевом почет,

Вога к за сто уго допавю рю.
А крашатыбрие, те вый к куголсл втоктьет за кросл, дроть
</s>
Канцет


100%|██████████| 126/126 [00:00<00:00, 458.97it/s]


    О вы, которые любовью не горели, вросточ,
Прит про тера в: рем нит ть ю в сялят ть италь ве,
О бориийсажде на лых усовнидезаяе е ртемот гисповенн ох, втынерой


100%|██████████| 331/331 [00:00<00:00, 407.11it/s]

    Все кончено: меж нами связи нет. иих ий!
Тлибох Меры.
Сма
В неднн,
Бря,
Веты всь шесвы нь клиши!
Пучтупенанимо мо ипедастетезыйт: в гавушастькта ксткадвозабьбебы кувдалу лье низаля всексарит гдое. овспролевный
Хренох маяснно,
Лю;
Прдевух;
Оль костам,
Алум…
Нимини с рали
Вый иве-детедазвиел пувогорв м бецашнаканометедогроро думел имеся, сы
И Крол, читьяти осконе


In [134]:
from sacrebleu import sentence_chrf

scores = [sentence_chrf(gen, references).score for gen in generated_texts]

print(scores)

[24.285954695523362, 34.75618231707053, 28.848892998956355]
